## 本章主要讲解以下内容：
- 介绍`PyTorch`的基本数据结构`Tensor`
- `Tensor`的操作
- 和`NumPy`的多维数组的交互
- `GPU`加快计算速度

进行机器学习和深度学习项目时，第一步是将外部输入转化成计算机表示，如浮点数等等，`PyTorch`提供了很好用的`Tensor`

# Tensor fundamentals

`Python`程序经常会处理矢量数据，通常情况下，`Python`使用列表存储,但是这不是最优方案，因为：
- `Python`中的`Numbers`是功能齐全的对象，`Python`将数字和属性打包成一个对象存储，数据量大时，会变得没有效率
- `Python`中的`List`存储对象集合，没有直接的向量点积操作，而且`list`是一维的，虽然可以列表的列表来表示多维，但是不高效
- `Python`语言相较于底层语言来说速度较慢

In [2]:
import torch
a = torch.ones(3)    #构建一个含有三个1的向量
a

tensor([1., 1., 1.])

In [5]:
a[1]    #访问向量的第二个量

tensor(1.)

In [6]:
float(a[1])    #可以进行强制类型转换

1.0

In [7]:
a[2] = 2.0    #改变第三个分量
a

tensor([1., 1., 2.])

以上操作虽然看上去和`list`没有太大区别，但是底层实现差别很大，`list`在内存中存储的是一个个对象，`tensor`存储的是数值.
构建`tensor`时，可以传入一个列表：

In [8]:
points = torch.tensor([1.0, 4.0, 2.0, 1.0, 3.0, 5.0])
points

tensor([1., 4., 2., 1., 3., 5.])

In [10]:
points = torch.tensor([[1.0, 4.0], [2.0, 1.0], [3.0, 5.0]])
points,points.shape

(tensor([[1., 4.],
         [2., 1.],
         [3., 5.]]), torch.Size([3, 2]))

也可以初始化一个固定结构的`tensor`，然后往里面填入数据:

In [12]:
points = torch.zeros(3, 2)
points

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [13]:
points[0, 1] = 1.0
points

tensor([[0., 1.],
        [0., 0.],
        [0., 0.]])

In [14]:
points[0]    #得到第一行的tensor

tensor([0., 1.])

### **注意**

上面返回第一行的`tensor`，并不是重新分配内存生成新的`tensor`，而是返回原来`tensor`第一行的`view`，这样数据量大的话，比较有效率

# Tensors and storages

`tensor`进行存储时，`value`存储在一段连续的内存中，由`torch.Storage`实例管理,`tensor`只是这一维存储空间中数据的`view`而已，不同的解读方法会产生不同的`tensor`

In [15]:
points = torch.tensor([[1.0, 4.0], [2.0, 1.0], [3.0, 5.0]])
points.storage()

 1.0
 4.0
 2.0
 1.0
 3.0
 5.0
[torch.FloatStorage of size 6]

这里，虽然points是一个`3*2`的`tensor`，但是在存储空间中，是一个一维的存储方式

In [16]:
points_storage = points.storage()
points_storage[0]

1.0

In [17]:
points.storage()[1]

4.0

In [18]:
points_storage[0] = 2.0   #改变存储空间中的值，points发生改变
points

tensor([[2., 4.],
        [2., 1.],
        [3., 5.]])

了解了`tensor`数据在内存中的存储方式，可以在你需要优化`pytorch`代码时发挥作用

# Size, storage offset, and strides

我们先看一下两个例子：

In [4]:
point1 = torch.tensor([[1, 2], [3, 4], [5, 6]])
point1_storage = point1.storage()
point1,point1_storage

(tensor([[1, 2],
         [3, 4],
         [5, 6]]),  1
  2
  3
  4
  5
  6
 [torch.LongStorage of size 6])

In [5]:
point2 = torch.tensor([[1, 2, 3], [4, 5, 6]])
point2_storage = point2.storage()
point2,point2_storage

(tensor([[1, 2, 3],
         [4, 5, 6]]),  1
  2
  3
  4
  5
  6
 [torch.LongStorage of size 6])

In [7]:
point1_storage[0] = 10
point1,point2            #改变其中一个，另一个不发生改变

(tensor([[10,  2],
         [ 3,  4],
         [ 5,  6]]), tensor([[1, 2, 3],
         [4, 5, 6]]))

以上代码我们可以看出，`point1`和`point2`的`storage`数值排布是一样的，但是可能不是同一个内存段。但是我们可以反过来考虑，同一段内存段使用不同的解读方法，是不是可以产生不一样的`tensor`对象。

所以如何从一段内存中解析出tensor成为了关键，我们需要引入以下几个量：
- **`size`** 指的是`tensor`每一维有多少元素，如`size=（3， 4）`说明有三行四列共12个元素
- **`storage offset`**  指的是`tensor`第一个元素对应内存中的`index`，指明了`tensor`映射内存块的起始`index`
- **`strides`**  指的是从当前维到下一维需要的偏移量，例如`points=[[1,2,3], [4,5,6]]`,从第一维的1到下一位相同位置4需要的偏移，反馈在`storage`里面就是`1，2，3，4，5，6`，从1到4所需的偏移，即3

In [8]:
dir(point1)        #可以查看point1自带属性方法

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_priority__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__contains__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__idiv__',
 '__ilshift__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pow__',
 '__radd__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rfloordiv__',
 '__rmul__',
 '__rpow__',
 '__rshift__',
 '__rsub__',
 '__rtruediv__',
 '__setattr__',
 '__se

In [12]:
point1.size()

torch.Size([3, 2])

In [13]:
point1.storage_offset()       #因为在storage第0个位置开始解析
#The offset will usually be zero; if this tensor is a view into a storage created to hold a larger tensor the offset might be a positive value.

0

In [14]:
point1.stride()     #这里2指的是跳到下一行需要+2，跳到下一列需要+1

(2, 1)

这里再提一下`subtensor`的概念：

In [15]:
points = torch.tensor([[1.0, 4.0], [2.0, 1.0], [3.0, 5.0]])
second_point = points[1]
points, second_point

(tensor([[1., 4.],
         [2., 1.],
         [3., 5.]]), tensor([2., 1.]))

这里`second_point`是`subtensor`,改变`second_point`，`points`跟着改变：

In [16]:
second_point[0] = 10.0
points

tensor([[ 1.,  4.],
        [10.,  1.],
        [ 3.,  5.]])

如果我们`clone`一个新的`tensor`,就不会发生上面的情况：

In [17]:
points = torch.tensor([[1.0, 4.0], [2.0, 1.0], [3.0, 5.0]])
second_point = points[1].clone()
second_point[0] = 10.0
points

tensor([[1., 4.],
        [2., 1.],
        [3., 5.]])

接下来说一下转置问题`transpose`:

In [18]:
points = torch.tensor([[1.0, 4.0], [2.0, 1.0], [3.0, 5.0]]) 
points

tensor([[1., 4.],
        [2., 1.],
        [3., 5.]])

In [19]:
points_t = points.t()
points_t

tensor([[1., 2., 3.],
        [4., 1., 5.]])

In [20]:
id(points.storage()) == id(points_t.storage())

True

由此可见，`tensor`的转置和原`tensor`share相同的`storage`,仅仅是双方的`shape`和`stride`不同

对于`points_t`来说，享有`points`相同的内存空间，但是对他来说就不是连续的了，读取需要在`storage`里反复`jumping`，影响效率。这里提供一个`contiguous`方法，可以使得转置后的`tensor`对应的`storage`符合它自己的数据分布：

In [28]:
points_t.storage()

 1.0
 4.0
 2.0
 1.0
 3.0
 5.0
[torch.FloatStorage of size 6]

In [29]:
points_t.stride()

(1, 2)

In [30]:
points_t_cont = points_t.contiguous()
points_t_cont

tensor([[1., 2., 3.],
        [4., 1., 5.]])

In [31]:
points_t_cont.stride()

(3, 1)

In [33]:
points_t_cont.storage()

 1.0
 2.0
 3.0
 4.0
 1.0
 5.0
[torch.FloatStorage of size 6]

可以看到使用`contiguous`方法后，`storage`发生了变化。

`Pytorch`中的转置不仅仅局限在`matrices`,我们甚至可以对多维数组进行转置，不过需要手动设置哪两维进行转置，看下面的例子：

In [21]:
some_tensor = torch.ones(3, 4, 5)
some_tensor_t = some_tensor.transpose(0, 2)  # 这里我们指定第一维和第三维转换，达到转置目的
some_tensor.shape

torch.Size([3, 4, 5])

In [22]:
some_tensor_t.shape

torch.Size([5, 4, 3])

In [23]:
some_tensor.stride()

(20, 5, 1)

In [24]:
some_tensor_t.stride()

(1, 5, 20)

# Numeric types

介绍完tensor创建和一些操作，我们还没有介绍tensor中数据类型问题，接下来简单总结tensor常用的数据类型：
- `torch.float32 or torch.float`—32-bit floating-point 
- `torch.float64 or torch.double`—64-bit, double-precision floating-point 
- `torch.float16 or torch.half`—16-bit, half-precision floating-point 
- `torch.int8`—Signed 8-bit integers 
- `torch.uint8`—Unsigned 8-bit integers 
- `torch.int16 or torch.short`—Signed 16-bit integers 
- `torch.int32 or torch.int`—Signed 32-bit integers 
- `torch.int64 or torch.long`—Signed 64-bit integers 

每一种数据类型都对应一个类，下面我们列出来：

|类型 | 类
|:-:|:-:
|torch.float|torch.FloatTensor or Tensor
|torch.double|torch.DoubleTensor
|torch.int8|torch.CharTensor
|torch.uint8|torch.ByteTensor

我们在创建`tensor`时可以指定数据类型,一般有四种方法：
```
1. double_points = torch.ones(10, 2, dtype=torch.double)
2. double_points = torch.zeros(10, 2).double() 
3. double_points = torch.zeros(10, 2).to(torch.double)
4. double_points = torch.zeros(10, 2).type(torch.double)
```

In [34]:
double_points = torch.ones(10, 2, dtype=torch.double)
double_points

tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]], dtype=torch.float64)

In [35]:
short_points = torch.tensor([[1, 2], [3, 4]], dtype=torch.short)
short_points

tensor([[1, 2],
        [3, 4]], dtype=torch.int16)

要查看一个`tensor`的类型，可以使用`dtype`属性：

In [36]:
short_points.dtype

torch.int16

In [40]:
#randn初始化0~1之间的数值
double_points = torch.randn(10, 2).type(torch.double)
double_points

tensor([[-1.6805, -0.9804],
        [ 0.7480,  1.0794],
        [ 0.1844,  1.5706],
        [-0.9566, -0.2911],
        [-0.2780,  0.1289],
        [-0.7939, -0.7016],
        [ 0.0873, -0.8903],
        [-1.2456,  0.7515],
        [ 0.2113,  0.8433],
        [ 0.6491,  1.2196]], dtype=torch.float64)

# Indexing tensors 

对应于列表中的切片操作，`Pytorch`中也有类似的操作：

In [41]:
points = torch.tensor([[1, 2], [3, 4], [5, 6]])
points

tensor([[1, 2],
        [3, 4],
        [5, 6]])

In [42]:
points[1:]        #从第二行到最后一行，所有列

tensor([[3, 4],
        [5, 6]])

In [43]:
points[1: , : ]   #同上

tensor([[3, 4],
        [5, 6]])

In [44]:
points[1: , 0]     #从第二行到最后一行，取第一列

tensor([3, 5])

In [45]:
points[ :-1, : -1]    #从第一行到倒数第二行，取第一列到倒数第二列

tensor([[1],
        [3]])

# NumPy interoperability

`Numpy`是数据科学的一个重要的工具，我们看看`pytorch`的`tensor`与`numpy`的`ndarray`之间如何转换：

In [46]:
points = torch.ones(3, 4)
points

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

In [47]:
points_np = points.numpy()
points_np

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)

**注意**

这里生成的`array`数组和原始`tensor`向量共享一片存储区域，修改其中一个，另一个也会发生变化。

In [48]:
points_np[0, 0] = 10
points              # 这里修改array，tensor发生变化

tensor([[10.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.]])

In [49]:
points[0, 1] = 20
points_np       # 这里修改tensor， array发生变化

array([[10., 20.,  1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.]], dtype=float32)

同样的，`array`也能转换成`tensor`：

In [50]:
points_1 = torch.from_numpy(points_np)
points_1

tensor([[10., 20.,  1.,  1.],
        [ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.]])

In [55]:
id(points.storage) == id(points_1.storage)    # 说明享有相同的缓冲区

True

In [57]:
id(points_np) == id(points_1)

False

In [53]:
points_1[0, 2] = 30
points_np          # 这里更新points_1, points_np 和 points都发生变化

array([[10., 20., 30.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.]], dtype=float32)

In [54]:
points

tensor([[10., 20., 30.,  1.],
        [ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.]])

# Serializing tensors

动态创建`tensor`很方便，但是有时候我们需要将创建的张量数据存储起来，以后用到直接加载即可。`pytorch`提供了底层基于`pickle`的序列化张量方法：
1. ```torch.save(points, './ourpoints.t')```
2. ```with open('./ourpoints.t', 'wb') as f:
      torch.save(points, f)```

In [58]:
points = torch.tensor([[1, 2], [3, 4], [5, 6]])
points

tensor([[1, 2],
        [3, 4],
        [5, 6]])

In [59]:
torch.save(points, './ourpoints.t')

或者使用下面的方法保存，两者等价。

In [60]:
with open('./ourpoints.t', 'wb') as f:
    torch.save(points, f)

我们尝试着从文件中导出原始数据：

In [61]:
points = torch.load('./ourpoints.t')
points

tensor([[1, 2],
        [3, 4],
        [5, 6]])

或者同样的：

In [62]:
with open('./ourpoints.t', 'rb') as f:
    points = torch.load(f)
points

tensor([[1, 2],
        [3, 4],
        [5, 6]])

# Moving tensors to the GPU

`pytorch`支持在`GPU`上对`tensor`进行数学操作，这样可以有效利用`GPU`的并行性操作特性，提高运行速度，那么我们如何将`tensor`移入GPU呢，又如何将处理好的数据转回到`CPU`呢，这就是我们接下来要说的内容：

将tensor传入GPU进行运算，有以下几种方法：
- `points_gpu = torch.tensor([[1.0, 4.0], [2.0, 1.0], [3.0, 4.0]], device='cuda')`   定义时就将变量定义在GPU上
- `points_gpu = points.to(device='cuda')`  使用to()方法将在CPU上创建的tensor复制一份到GPU中
- `points_gpu = points.to(device='cuda:0')`  拥有多个GPU时，可以选择复制到哪个GPU上
- `points_gpu = points.cuda()`    默认GPU索引为0
- `points_gpu = points.cuda(0)` 

**补充**
存入`GPU`的`tensor`，数据类型变成`torch.cuda.FloatTensor`或者其他

In [65]:
torch.cuda.is_available()       # 通过这条语句判断有无cuda环境

False

tensor在GPU上运行的流程：
```
1. The points tensor was copied to the GPU. 
2. A new tensor was allocated on the GPU and used to store the result of the multiplication. 
3. A handle to that GPU tensor was returned. 
```

In [ ]:
points = torch.tensor([[1, 2], [3, 4], [5, 6]])   #在CPU上创建一个tensor
points_gpu = points.to(device='cuda:0')          #将tensor复制到GPU中
points_gpu = 2 * points_gpu + 4                  #对tensor进行科学运算
points_cpu = points_gpu.to(device='cpu')        #将GPU上运算好的tensor在重新转移到CPU内，进行其他操作

# The tensor API

这里我们列举一些`pytorch`提供的关于`tensor`的操作，完整操作列表请查阅[官方文档](http://pytorch.org/docs."官方文档")

转置：

In [66]:
a = torch.ones(3, 2)
a_t = torch.transpose(a, 0, 1)          # 这里表示第0维和第一维交换，达到转置效果
# 或者换一种写法
# a_t = a.transpose(0, 1)
a, a_t

(tensor([[1., 1.],
         [1., 1.],
         [1., 1.]]), tensor([[1., 1., 1.],
         [1., 1., 1.]]))

类似乎python语法，方法带下划线的会覆盖掉输入变量，不带下划线会新建一个变量，不改变原来变量

In [67]:
a = torch.ones(3, 2)
a

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])

In [68]:
a.zero_()
a

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

上面提到的官方文档，主要介绍以下部分：
- *`Creation ops`*—Functions for constructing a tensor, such as `ones` and `from_numpy` 
- *``Indexing`, `slicing`, `joining`, and `mutating ops``*—Functions for changing the shape, stride, or content of a tensor, such as `transpose`
- *`Math ops`*—Functions for manipulating the content of the tensor through computations: 
    > *`Pointwise ops`*—Functions for obtaining a new tensor by applying a function to each element independently, such as `abs` and `cos`.<p>
    > *`Reduction ops`*—Functions for computing aggregate values by iterating through tensors, such as `mean`, `std`, and `norm`.<p>
    > *`Comparison ops`*—Functions for evaluating numerical predicates over tensors, such as `equal` and `max` .<p>
    > *`Spectral ops`*—Functions for transforming in and operating in the frequency domain, such as `stft` and `hamming_window` .<p>
    > *`Other ops`*—Special functions operating on vectors, such as cross, or matrices, such as `trace `.<p>
    > *`BLAS and LAPACK ops`*—Functions that follow the BLAS (Basic Linear Algebra Subprograms) specification for scalar, vector-vector, matrix-vector, and matrix-matrix operations.<p>
- *`Random sampling ops`*—Functions for generating values by drawing randomly from probability distributions, such as `randn` and `normal` 
- *`Serialization ops`*—Functions for saving and loading tensors, such as `load` and `save` 
- *`Parallelism ops`*—Functions for controlling the number of threads for parallel CPU execution, such as `set_num_threads `